In [24]:
import gsd.hoomd
import numpy as np
import gsd.pygsd
import hoomd
import matplotlib

In [25]:
%matplotlib inline
matplotlib.style.use('ggplot')
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [13]:
frame = gsd.hoomd.Frame()
frame.particles.N = 10
frame.particles.types = ['A','M','U']
frame.particles.type = np.random.randint(0,len(frame.particles.types),frame.particles.N)
frame.particles.position = np.random.randint(0, high=frame.particles.N, size=(frame.particles.N,3), dtype=int)


In [14]:
f = gsd.hoomd.open(name='file.gsd', mode='w')

In [15]:
def create_frame(i):

    frame = gsd.hoomd.Frame()

    frame.configuration.step = i

    return frame


f = gsd.hoomd.open(name='example.gsd', mode='w')

f.extend( (create_frame(i) for i in range(10)) )

f.append( create_frame(10) )

In [16]:
# Python reading function
f = gsd.pygsd.GSDFile(open('example.gsd', 'rb'))

trajectory = gsd.hoomd.HOOMDTrajectory(f)

trajectory.particles.position

AttributeError: 'HOOMDTrajectory' object has no attribute 'particles'

In [26]:
integrator = hoomd.md.Integrator(dt=0.005)

In [27]:
integrator.forces[:]

[]

In [28]:
def potential_energy(positions, bonds, l0, b):
    repulsion_term = np.sum(np.exp(-4 * np.abs(positions[:, np.newaxis] - positions) / l0), dtype=np.float128)
    bonding_term = np.sum(np.exp(-4 * b * np.abs(positions[:, np.newaxis] - positions[bonds]) / l0), dtype=np.float128)
    #
    return repulsion_term - bonding_term

In [29]:
cell = hoomd.md.nlist.Cell(buffer=0.4)

In [ ]:
exp_potential = hoomd.md.pair.Table(nlist = cell,)